In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException 
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd

In [120]:
driver = webdriver.Firefox()
#last month
driver.get("https://inspections.myhealthdepartment.com/sacramento")

wait = WebDriverWait(driver, 5)

date_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#filterdate')))
date_button.click()

for i in range(10):
    prev_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div[2]/div[1]/table/thead/tr[1]/th[1]')))
    prev_button.click()

begin_date = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[4]")))
begin_date.click()

for i in range(8):
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div[3]/div[1]/table/thead/tr[1]/th[3]')))
    next_button.click()

end_date = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[4]/div[3]/div[1]/table/tbody/tr[6]/td[1]")))
end_date.click()

apply = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[4]/div[4]/button[2]")))
apply.click()

while True:
    try:
        load_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[9]/div/button')))
        load_more_button.click()
        time.sleep(1)
    except TimeoutException:
        print('Timeout Exception, Stopping')
        break

view_buttons = driver.find_elements(By.XPATH, '//a[contains(text(), "View")]')

links = [button.get_attribute('href') for button in view_buttons]

driver.quit()

Timeout Exception, Stopping


In [124]:
import pickle

with open("Data/Health2.pkl", "ab") as f:
    pickle.dump(links, f)

In [87]:
import pickle
with open("Data/Health.pkl", "rb") as f:
    links = pickle.load(f)
len(links)

2268

In [3]:
import json

with open("Data/combined_results.json", 'r') as f:
    health_codes = json.load(f)

with open("Data/yelp_cleaned.json", 'r') as f:
    yelp_reviews = json.load(f)

In [8]:
health_codes
columns = ["name", "address", "health codes", "date", "url"]

df_health = pd.DataFrame(health_codes, columns=columns)

In [13]:
df_health

df_health["key"] = df_health.apply(lambda row: row['address'].split()[0][:4] + "-" + row['name'][:3].lower(), axis=1)

In [15]:
df_health.to_csv("Data/healthdf.csv")

In [22]:
yelp_reviews

{'4701-tha': {'alias': 'thai-the-house-of-authentic-ingredients-sacramento',
  'name': 'THAI - The House Of Authentic Ingredients',
  'review_count': 571,
  'categories': ['Thai', 'Bars'],
  'display_address': ['4701 H St', 'Sacramento, CA 95819'],
  'addy_one': '4701 H St',
  'addy_two': None,
  'addy_three': None,
  'city': 'Sacramento',
  'zip_code': '95819',
  'phone': '+19169429008'},
 '4601-vib': {'alias': 'vibe-health-bar-sacramento-2',
  'name': 'Vibe Health Bar',
  'review_count': 322,
  'categories': ['Juice Bars & Smoothies', 'Wraps', 'Sandwiches'],
  'display_address': ['4601 H St', 'Sacramento, CA 95819'],
  'addy_one': '4601 H St',
  'addy_two': '',
  'addy_three': '',
  'city': 'Sacramento',
  'zip_code': '95819',
  'phone': '+19169705102'},
 '160-ber': {'alias': 'bertha-henschel-park-sacramento',
  'name': 'Bertha Henschel Park',
  'review_count': 17,
  'categories': ['Parks', 'Playgrounds'],
  'display_address': ['160 45th St', 'Sacramento, CA 95819'],
  'addy_one': '1

In [25]:
key_list = []
for key in yelp_reviews.keys():
    key_list.append(key)

In [28]:
yelp_reviews[key_list[0]]
key_list[0]

'4701-tha'

In [31]:
data_list = [{'key': key, **value} for key, value in yelp_reviews.items()]

df_yelp = pd.DataFrame(data_list)

In [36]:
df_yelp.to_csv("Data/yelpdf.csv")